<img src="Imagenes/Mac_wallpaper_3.png" width="50%">

# Algoritmo de Deutsch-Jozsa

El algoritmo de Deutsch-Jozsa fue el primer ejemplo de un algoritmo cuántico que posee una ventaja sobre el mejor algoritmo clásico. Y aunque no tenga una aplicación útil como tal, es un buen ejemplo de que utilizar una computadora cuántica en lugar de una clásica puede ser ventajoso para ciertos problemas. 

### El problema de Deutsch-Jozsa

Nosotros tenemos una función misteriosa $f$ (la cual no conocemos como opera) que recibe como entrada una cadena de $n$ bits y regresa un $0$ o un $1$

$\hspace{7 cm} f(x_{n-1}x_{n-2}\cdots x_{1}x_{0}) \rightarrow 0 \hspace{0.2 cm} o \hspace{0.2 cm} 1 \hspace{0.5 cm}$ donde $\hspace{0.5 cm} x_{i} \in \{0,1\}$

Sin embargo, sabemos con seguridad que la función es constante o balanceada. <br>
- Si es constante regresará solo 0 o solo 1 para todos los inputs.
- Si es balanceada regresará 0 para la mitad de los inputs y 1 para la otra mitad.

Nuestra tarea es determinar si la función $f$ es balanceada o constante, siendo que la única forma que tenemos de extraer información de la función es aplicándola.

### El caso clásico

Si fuéramos a resolver este problema de manera clásica tenemos que en el mejor de los casos (si tenemos mucha suerte) podemos saber si la función es constante o balanceada haciendo únicamente dos llamados a la función. Por ejemplo, si observamos que $f(100) = 1$ y $f(010) = 0$ sabremos con total seguridad que la función es balanceada.

Ahora, en el peor de los casos (en el que siempre obtenemos el mismo valor como respuesta) no podremos estar seguros al 100% de si la función es balanceada o constante hasta que hagamos $N/2 + 1$ llamadas a la función, donde $N$ es el número de cadenas de bits que tenemos como entrada. Y si $n$ es el número de bits en nuestro sistema, entonces $N = 2^{n}$, por lo que necesitaríamos llamar a nuestra función $2^{n-1}+1$ veces.

Por ejemplo, si nuestro sistema es de 5 bits, necesitaríamos hacer un total de 17 operaciones para estar 100% seguros de nuestra respuesta, pues es posible que aunque las primeras 16 operaciones dieran todas $0$ la número 17 de como resultado $1$, significando que la función es balanceada.

Obviamente estamos hablando del peor escenario posible, y en teoría podríamos truncar nuestro proceso antes de llegar a esas $2^{n-1}+1$ operaciones dependiendo de que tanto margen de error querramos; pero la única forma de estar 100% seguros es haciendo $2^{n-1}+1$ operaciones. Es decir, tenemos un algoritmo cuyo tiempo de ejecución crece exponencialmente con el número de bits.

### ¿Cómo implementar nuestra función en un circuito cuántico?

Antes de dar la solución cuántica a este problema, es necesario hallar una forma de implementar la función $f$ como un operador que:

- Sea unitario (es decir, que deje igual la norma de un estado luego de aplicarse)
- Sea reversible (aplicando un operador al estado final podemos regresar al estado inicial siempre)

Con estos dos requisitos en mente, vamos a implementar la función $f$ como un operador $U_{f}$ que actúe de la siguiente manera:

$\hspace{10 cm} U_{f}|x\rangle|y\rangle = |x\rangle|y \oplus f(x)\rangle$

Donde $x$ es la cadena de bits de tamaño $n$ que nuestra función recibe como input, $y$ es un estado cualquiera (del mismo tamaño que la cadena de bits $f(x)$) que servirá para almacenar el output y el símbolo $\oplus$ representa la suma módulo 2 bit por bit

| Operación | Resultado |
| :---: | :---: |
| $0\oplus 0$ | 0 |
| $0\oplus 1$ | 1 |
| $1\oplus 0$ | 1 |
| $1\oplus 1$ | 0 |

De esta forma, podemos observar que si aplicamos nuestro operador una segunda vez recuperamos nuestro estado inicial, ya que

$\hspace{8 cm} U_{f}|x\rangle|y\oplus f(x)\rangle = |x\rangle|y \oplus f(x) \oplus f(x)\rangle = |x\rangle|y\rangle$

Pues $f(x) \oplus f(x) = 0$ independientemente del valor de $f(x)$ (puedes comprobarlo viendo a la tabla de arriba) e $y \oplus 0 = y$ sin importar el valor de $y$.

Ya con la implementación de nuestra función decidida, es momento de hablar de la solución cuántica.

### El caso cuántico

En este caso, necesitamos un circuito con $n+1$ qubits. Los primeros $n$ qubits (nuestro input) se inicializan en el estado $|0\rangle$ y el último qubit se inicializa en el estado $|1\rangle$ (pronto veremos por qué). A continuación presentamos cómo luce el circuito para resolver el problema de Deutsch-Jozsa (extraída del textbook de Qiskit)

<img src="Imagenes/DJ.png" width="60%">

Ahora, veamos algo de notación. Esta la utilizaremos con los demás algoritmos, por lo que es necesario que quede clara desde el inicio.

- El término $|0^{\otimes n}\rangle$ se refiere al estado formado por $n$ qubits, todos en el estado $|0\rangle$. Es decir $|0^{\otimes n}\rangle = |0\rangle_{n-1}\otimes \cdots \otimes|0\rangle_{0}$.
- El término $H^{\otimes n}$ se refiere al operador formado por el producto tensorial de $n$ compuertas Hadamard. Es decir $H^{\otimes n} = H_{n-1}\otimes \cdots \otimes H_{0}$.

Ahora sí, veamos cómo funciona el algoritmo paso por paso:

*Nota: A partir de aquí hay mucha matemática para explicar cómo funciona el algoritmo. No es necesario que lo comprendas todo a la perfección, solo que te convenzas de que el algoritmo funciona.*

---

**Primer paso**: Aplicar una compuerta Hadamard a todos los qubits. Este paso es trivial para el qubit del output, pues sabemos que

$\hspace{10 cm} H|1\rangle = \dfrac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$

En el caso del estado $|0^{\otimes n}\rangle$ solo debemos generalizar el resultado de aplicar compuertas Hadamard a todos los qubits: <br>
- Para $n=1$: $H|0\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$.
- Para $n=2$: $H\otimes H|00\rangle = \frac{1}{\sqrt{2^{2}}}(|00\rangle + |01\rangle + |10\rangle + |11\rangle)$.

Si seguimos aumentando el valor de $n$ veremos que el resultado es una superposición equitativa (todos los estados tienen la misma amplitud) de todas las posibles $2^{n}$ diferentes combinaciones de los estados de los $n$ qubits, la cual se puede expresar de la siguiente forma:

$\hspace{10 cm} H^{\otimes n}|0^{\otimes n}\rangle = \dfrac{1}{\sqrt{2^{n}}}\displaystyle\sum_{x=0}^{2^{n}-1}|x\rangle$

*Nota: Si por ejemplo tenemos $n=2$ qubits, la sumatoria sería $|0\rangle + |1\rangle + |2\rangle + |3\rangle$, donde estos estados son los estados base pero escritos como enteros en lugar de en binario. Es decir $|0\rangle = |00\rangle$, $|1\rangle = |01\rangle$, $|2\rangle = |10\rangle$ y $|3\rangle = |11\rangle$.*

Por lo que el estado del sistema completo antes de la primera barrera sería:

$\hspace{9 cm} |\psi_{1}\rangle = \dfrac{1}{\sqrt{2^{n+1}}}\displaystyle\sum_{x=0}^{2^{n}-1}|x\rangle(|0\rangle - |1\rangle)$

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer

#Creamos dos registros cuánticos, uno para la entrada y el otro para almacenar la salida
qin = QuantumRegister(3, name="in") #En este caso, n=3
qout = QuantumRegister(1, name="out")

#Juntamos ambos registros en un solo circuito
qc = QuantumCircuit(qin,qout)

#Preparamos el qubit del output en el estado |->
qc.x(qout[0])
qc.h(qout[0])

#Aplicamos las compuertas Hadamard a todos los qubits del input
for i in range(3):
    qc.h(qin[i])
    
#Dibujamos nuestro circuito
qc.draw(output="mpl")

---

**Segundo paso**: Aplicamos el operador $U_{f}$:

$\hspace{7.5 cm} |\psi_{2}\rangle = U_{f}|\psi_{1}\rangle = \dfrac{1}{\sqrt{2^{n+1}}}\displaystyle\sum_{x=0}^{2^{n}-1}|x\rangle(|0\oplus f(x)\rangle - |1 \oplus f(x)\rangle)$

Ya que $f(x)$ solo puede tomar dos valores, tenemos dos opciones:<br>
- Si $f(x) = 0$ entonces el estado del qubit del output se queda como $|0\rangle - |1\rangle$.
- Si $f(x) = 1$ entonces el estado del qubit del output cambia a $|1\rangle - |0\rangle = -(|0\rangle - |1\rangle)$

Esto lo podemos resumir como $(-1)^{f(x)}(|0\rangle - |1\rangle)$. De esta forma, nuestro estado queda como

$\hspace{9 cm} |\psi_{2}\rangle = \dfrac{1}{\sqrt{2^{n+1}}}\displaystyle\sum_{x=0}^{2^{n}-1}(-1)^{f(x)}|x\rangle(|0\rangle - |1\rangle)$

Utilizando el retroceso de fase hemos sido capaces de "pasar" el efecto del operador $U_{f}$ del qubit del output a nuestra cadena de qubits del input, en forma de una fase local (pues depende del valor de $f(x)$). Es por esto que inicializamos el qubit del output como $|1\rangle$ antes de aplicar una compuerta Hadamard, pues si lo hubiéramos inicializado como $|0\rangle$ no tendríamos el signo menos que es el cuasante del retroceso de fase. Esto nos permite ignorar el estado del qubit del output a partir de este punto, pues ya no usaremos más compuertas sobre este.

Para este ejemplo usaremos una función constante que siempre regresa **1** como su resultado. Para implementarla basta con aplicar una compuerta NOT al qubit del output.

In [ ]:
#Creamos dos registros cuánticos, uno para la entrada y el otro para almacenar la salida
qin = QuantumRegister(3, name="in") #En este caso, n=3
qout = QuantumRegister(1, name="out")

#Juntamos ambos registros en un solo circuito
qc = QuantumCircuit(qin,qout)

#Preparamos el qubit del output en el estado |->
qc.x(qout[0])
qc.h(qout[0])

#Aplicamos las compuertas Hadamard a todos los qubits del input
for i in range(3):
    qc.h(qin[i])
    
qc.barrier()

#Aplicamos el operador constante U_f
qc.x(qout[0])
    
#Dibujamos nuestro circuito
qc.draw(output="mpl")

---

**Tercer paso**: Aplicar compuertas Hadamard a todos los qubits del input. Para ello ahora debemos generalizar lo que sucede al aplicar el operador $H^{\otimes n}$ a un estado $|a\rangle$ arbitrario.

Primero, recordemos que la única diferencia entre aplicar el operador Hadamard a $|0\rangle$ o a $|1\rangle$ es un signo menos:

$\hspace{10 cm} H|0\rangle = \dfrac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$

$\hspace{10 cm} H|1\rangle = \dfrac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$

De esta forma, si $a$ es una cadena de un solo bit (es decir, los únicos valores que puede tomar son $0$ y $1$), podemos escribir el resultado de aplicarle una compuerta Hadamard como

$\hspace{10 cm} H|a\rangle = \dfrac{1}{\sqrt{2}}\displaystyle\sum_{y=0}^{1}(-1)^{a\cdot y}|y\rangle$

En este caso si $y = 0$ siempre va a estar multiplicado por $1$, independientemente del valor de $a$. Pero si $y= 1$ y $a = 1$, tenemos un factor de $-1$ correspondiente a nuestro estado $|-\rangle$. 

En el caso de que $a$ sea una cadena de dos bits tenemos cuatro posibilidades:

$\hspace{9 cm} H^{\otimes 2}|00\rangle = \dfrac{1}{2}(|00\rangle + |01\rangle + |10\rangle + |11\rangle)$

$\hspace{9 cm} H^{\otimes 2}|01\rangle = \dfrac{1}{2}(|00\rangle - |01\rangle + |10\rangle - |11\rangle)$

$\hspace{9 cm} H^{\otimes 2}|10\rangle = \dfrac{1}{2}(|00\rangle + |01\rangle - |10\rangle - |11\rangle)$

$\hspace{9 cm} H^{\otimes 2}|11\rangle = \dfrac{1}{2}(|00\rangle - |01\rangle - |10\rangle + |11\rangle)$

Podemos condensar estos cuatro posibles casos en la siguiente expresión:

$\hspace{10 cm} H^{\otimes 2}|a\rangle = \dfrac{1}{2}\displaystyle\sum_{y=0}^{3}(-1)^{a\cdot y}|y\rangle$

Donde, escribiendo $a=a_{1}a_{0}$ e $y=y_{1}y_{0}$ (por ejemplo, si $a=01$ e $y= 10$ entonces $a_{1}=0,a_{0}=1,y_{1}=1,y_{0}=0$), tenemos que $a\cdot y = a_{1}y_{1} \oplus a_{0}y_{0}$. Puedes comprobar por tu cuenta que usando esta expresión podemos reproducir los cuatro resultados de arriba.

Entonces, si generalizamos esto a $n$ qubits obtenemos la expresión

$\hspace{9 cm} H^{\otimes n}|a\rangle = \dfrac{1}{\sqrt{2^{n}}}\displaystyle\sum_{y=0}^{2^{n}-1}(-1)^{a\cdot y}|y\rangle$

Donde $a\cdot y = a_{n-1}y_{n-1}\oplus \cdots \oplus a_{1}y_{1} \oplus a_{0}y_{0}$. <br><br>
*Nota: Si $|a\rangle = |0^{\otimes n}\rangle$ entonces $a\cdot y = 0$ sin importar el valor de $y$, por lo que recuperamos el primer caso que vimos.*

Con ayuda de esta nueva expresión podemos calcular el nuevo estado del sistema luego de aplicar las compuertas Hadamard (vamos a ignorar el estado del qubit del output ya que podemos separarlo utilizando el producto tensorial).

$\hspace{7 cm} |\psi_{3}\rangle = H^{\otimes n}|\psi_{2}\rangle = \dfrac{1}{\sqrt{2^{n}}}\displaystyle\sum_{x=0}^{2^{n}-1}(-1)^{f(x)}\left(\dfrac{1}{\sqrt{2^{n}}}\displaystyle\sum_{y=0}^{2^{n}-1}(-1)^{x\cdot y}|y\rangle\right)$

Notamos que tenemos dos sumatorias: la sumatoria sobre $y$ es una sumatoria sobre todos los estados de nuestra superposición, y la sumatoria sobre $x$ nos da la amplitud de cada uno de los estados $|y\rangle$, por lo que podemos reordenar la expresión de arriba como

$\hspace{9 cm} |\psi_{3}\rangle = \displaystyle\sum_{y=0}^{2^{n}-1}\left(\dfrac{1}{2^{n}}\displaystyle\sum_{x=0}^{2^{n}-1}(-1)^{f(x)}(-1)^{x\cdot y}\right)|y\rangle$

In [ ]:
#Creamos dos registros cuánticos, uno para la entrada y el otro para almacenar la salida
qin = QuantumRegister(3, name="in") #En este caso, n=3
qout = QuantumRegister(1, name="out")

#Juntamos ambos registros en un solo circuito
qc = QuantumCircuit(qin,qout)

#Preparamos el qubit del output en el estado |->
qc.x(qout[0])
qc.h(qout[0])

#Aplicamos las compuertas Hadamard a todos los qubits del input
for i in range(3):
    qc.h(qin[i])
    
qc.barrier()

#Aplicamos el operador constante U_f
qc.x(qout[0])

qc.barrier()

#Aplicamos las compuertas Hadamard a todos los qubits del input nuevamente
for i in range(3):
    qc.h(qin[i])
    
#Dibujamos nuestro circuito
qc.draw(output="mpl")

---

**Último paso**: Medir el registro del input.

Es aquí donde entra en juego la ventaja del algoritmo cuántico ¿Recuerdas cómo en la versión clásica necesitábamos llamar a la función $2^{n-1}+1$ veces para estar 100% seguros de si la función es constante o balanceada? Bueno, en el caso cuántico podemos estar 100% de si la función es constante o balanceada ¡llamando solo una vez a la función!

Para ver cómo es esto posible, vamos a calcular la probabilidad de medir la cadena de solo ceros al momento de medir el registro del input, es decir, la probabilidad de que $|y\rangle = |0^{\otimes n}\rangle$. Después de todo, al hacer la medición forzamos al estado $|\psi_{3}\rangle$ a colapsar a alguno de los posibles estados $|y\rangle$. En este caso $x\cdot y = 0$ para cualquier cadena de bits $x$, por lo que la probabilidad se reduce a

$\hspace{11 cm} \left|\dfrac{1}{2^{n}}\displaystyle\sum_{x=0}^{2^{n}-1}(-1)^{f(x)}\right|^{2}$

En este caso, tenemos dos posibilidades:<br>
- Si la función es balanceada, tenemos igual cantidad de $1$ y $-1$ en la sumatoria, por lo que se cancelan perfectamente entre ellos y la probabilidad es igual a **0**.
- Si la función es constante, tenemos o solo $1$ o solo $-1$ en la sumatoria, y ya que hay $2^{n}$ términos dentro de esta el resultado es $2^{n}$ o $-2^{n}$. Por lo tanto, la probabilidad es igual a **1**.

De esta manera, si el resultado de la medición es la cadena de solo ceros la función es constante; si es cualquier otra cadena de bits la función es balanceada.

In [ ]:
#Creamos dos registros cuánticos, uno para la entrada y el otro para almacenar la salida
qin = QuantumRegister(3, name="in") #En este caso, n=3
qout = QuantumRegister(1, name="out")

#Creamos un registro clásico para almacenar el resultado de la medición
c = ClassicalRegister(3)

#Juntamos ambos registros en un solo circuito
qc = QuantumCircuit(qin,qout,c)

#Preparamos el qubit del output en el estado |->
qc.x(qout[0])
qc.h(qout[0])

#Aplicamos las compuertas Hadamard a todos los qubits del input
for i in range(3):
    qc.h(qin[i])
    
qc.barrier()

#Aplicamos el operador constante U_f
qc.x(qout[0])

qc.barrier()

#Aplicamos las compuertas Hadamard a todos los qubits del input nuevamente
for i in range(3):
    qc.h(qin[i])
    
qc.barrier()

#Medimos el registro del input
qc.measure(qin,c)
    
#Dibujamos nuestro circuito
qc.draw(output="mpl")

In [ ]:
#Ejecutamos el circuito solo una vez
job = execute(qc,Aer.get_backend('qasm_simulator'),shots=1)
counts = job.result().get_counts(qc)
print(counts)

Como podemos ver, el resultado de la medición es la cadena de solo ceros, por lo que la función es constante.

---

### ¿Cómo codificar una función constante?

Como un pequeño extra, vamos a mostrar cómo podemos codificar una función constante y una balanceada, para que puedas experimentar más con el algoritmo de Deutsch-Jozsa. Primero, veremos como codificar una función constante, pues como hemos visto es bastante sencillo.

Para ello, vamos a suponer que el registro del output se encuentra en el estado $|0\rangle$ para que los cálculos sean más sencillos (si el output se encuentra en cualquier otro estado no importa, ya que el retroceso de fase nos permite pasar el efecto de la función al registro del input).

De esta manera, si nuestra función tiene como output el valor $0$ dejamos el qubit del output tal y como está, y si nuestra función tiene como output el valor $1$ aplicamos una compuerta NOT al output. Podemos usar la función `randrange()` para generar un número aleatorio entre 0 y 1 para saber qué caso aplicar.

In [ ]:
from random import randrange

#Definimos una función que tome como parámetros un circuito cuántico, un registro cuántico y un número aleatorio entre 0 y 1
def constante(qc,qout,x):
    
#Si el valor es igual a 1
    if x == 1:
#Aplicamos una compuerta NOT al output
        qc.x(qout[0])

In [ ]:
estados = ["000","001","010","011","100","101","110","111"]

#Generamos un número aleatorio entre 0 y 1
x = randrange(2)

for estado in estados:
    
    qin = QuantumRegister(3, name="in")
    qout = QuantumRegister(1, name="out")

#Creamos un registro clásico para medir el output
    c = ClassicalRegister(1)
    
    qc = QuantumCircuit(qin,qout,c)

#Codificamos el estado de los inputs
    if int(estado[0]) == 1:
        qc.x(qin[2])
    if int(estado[1]) == 1:
        qc.x(qin[1])
    if int(estado[2]) == 1:
        qc.x(qin[0])
        
#Aplicamos la función constante
    constante(qc,qout,x)
    
#Medimos el qubit del output
    qc.measure(qout,c)
    
#Ejecutamos nuestro circuito
    job = execute(qc,Aer.get_backend('qasm_simulator'),shots=1)
    counts = job.result().get_counts(qc)
    print("Input:",estado,"Output:",counts)

### ¿Cómo codificar una función balanceada?

El caso de la función balanceada es un poco más complejo. En este caso queremos que en la mitad de los inputs se aplique una compuerta NOT al output y en la otra mitad no se aplique nada. De esta forma, la mitad de los outputs serán $0$ y la otra mitad serán $1$.

Esto lo podemos conseguir utilizando compuertas CNOT entre todos los qubits del input y el qubit del output, tomando al qubit del output como el objetivo. Esta estrategia funciona ya que aplicar un número par de compuertas NOT en el qubit del output dejará su estado sin alterar (es decir, lo dejará en $|0\rangle$) mientras que un número impar de compuertas NOT cambiará su estado (es decir, lo cambiará a $|1\rangle$). Solo falta convencerte de que la mitad de las cadenas de $n$ bits tienen un número par de $1$ y la otra mitad tiene un número impar de $1$.

In [ ]:
#Creamos una función que toma como parámetros un circuito cuántico, el registro del input, el del output, y el número de qubits
#en el input
def balanceada(qc,qin,qout,n):

#Creamos un bucle para aplicar una compuerta CNOT entre cada qubit del input y el qubit del output
        for i in range(n):
            qc.cx(qin[i],qout[0])

In [ ]:
estados = ["000","001","010","011","100","101","110","111"]

for estado in estados:
    
    qin = QuantumRegister(3, name="in")
    qout = QuantumRegister(1, name="out")

#Creamos un registro clásico para medir el output
    c = ClassicalRegister(1)
    
    qc = QuantumCircuit(qin,qout,c)

#Codificamos el estado de los inputs
    if int(estado[0]) == 1:
        qc.x(qin[2])
    if int(estado[1]) == 1:
        qc.x(qin[1])
    if int(estado[2]) == 1:
        qc.x(qin[0])
        
#Aplicamos la función balanceada
    balanceada(qc,qin,qout,3)
    
#Medimos el qubit del output
    qc.measure(qout,c)
    
#Ejecutamos nuestro circuito
    job = execute(qc,Aer.get_backend('qasm_simulator'),shots=1)
    counts = job.result().get_counts(qc)
    print("Input:",estado,"Output:",counts)

Ahora solo resta que te pongas a experimentar con estas funciones para ver cómo funciona el algoritmo de Deutsch-Jozsa en general.

In [ ]:
#Creamos dos registros cuánticos, uno para la entrada y el otro para almacenar la salida
qin = QuantumRegister(3, name="in") #En este caso, n=3
qout = QuantumRegister(1, name="out")

#Creamos un registro clásico para almacenar el resultado de la medición
c = ClassicalRegister(3)

#Juntamos ambos registros en un solo circuito
qc = QuantumCircuit(qin,qout,c)

#Preparamos el qubit del output en el estado |->
qc.x(qout[0])
qc.h(qout[0])

#Aplicamos las compuertas Hadamard a todos los qubits del input
for i in range(3):
    qc.h(qin[i])
    
qc.barrier()

#Aplicamos nuestra función balanceada o constante
balanceada(qc,qin,qout,3)
#constante(qc,qout,1)

qc.barrier()

#Aplicamos las compuertas Hadamard a todos los qubits del input nuevamente
for i in range(3):
    qc.h(qin[i])
    
qc.barrier()

#Medimos el registro del input
qc.measure(qin,c)
    
#Dibujamos nuestro circuito
qc.draw(output="mpl")

In [ ]:
#Ejecutamos el circuito solo una vez
job = execute(qc,Aer.get_backend('qasm_simulator'),shots=1)
counts = job.result().get_counts(qc)
print(counts)